In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

from oneDtoyTask.oneDtoyTask import ToyTask, OneDfield, TwoWayComChannel
from agents.agent_Aria import Aria

from utils import getReturns, Controller

In [ ]:
batch_size = 1
agent0 = Aria(batch_size=batch_size)
agent1 = Aria(batch_size=batch_size)

learningRate = 0.0005 
epochs = 100
optimizer1 = torch.optim.Adam(agent0.parameters(), lr=learningRate)
optimizer2 = torch.optim.Adam(agent1.parameters(), lr=learningRate)
np.random.seed(1)
field = OneDfield(speed=10)
Task = ToyTask(field=field, observationMappingFct=lambda x: (x>0.5).astype(int), comChannel=TwoWayComChannel())



In [ ]:
obs, downlink_msgs = Task.reset()
C = Controller()
for epoch in range(epochs):
    a0p, uplink_m0 = agent0(obs[0], downlink_msgs[0])
    a1p, uplink_m1 = agent1(obs[1], downlink_msgs[1])
    logp_a0 = torch.log(a0p)
    logp_a1 = torch.log(a1p)
    logp_uplink0 = torch.log(uplink_m0)
    logp_uplink1 = torch.log(uplink_m1)
    actions.append((a0p, a1p))
    states.append((obs, downlink_msgs))
    (obs, downlink_msgs), r, done = Task.step([a0, a1], [uplink_m0, uplink_m1])
    C.insert_traj((obs, downlink_msgs), r, [a0p.to_numpy(), a1p.to_numpy()], )
    rewards.append(r)

    